In [1]:
import pandas as pd
import numpy as np
import statsmodels as stats
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import scipy 
from statsmodels.tsa.stattools  import   grangercausalitytests

print('pandas: %s' % pd.__version__)
print('numpy: %s' % np.__version__)
print('statsmodels: %s' % stats.__version__)
print('tensorflow: %s' % tf.__version__)
print('matplotlib: %s' % matplotlib.__version__)
print('scipy: %s' % scipy.__version__)

pandas: 0.18.1
numpy: 1.13.1
statsmodels: 0.8.0.dev0+302c2ec
tensorflow: 1.2.0
matplotlib: 2.0.0
scipy: 0.18.1


In [2]:
# input the data
amb_temp = pd.read_csv("amb_temp.csv")
comp_temp = pd.read_csv("comp_temp.csv")
mcu_temp = pd.read_csv("mcu_temp.csv")
current = pd.read_csv("current.csv")
door = pd.read_csv("door.csv")
energy = pd.read_csv("energy.csv")
power = pd.read_csv("power.csv")
power_factor = pd.read_csv("power_factor.csv")
voltage = pd.read_csv("voltage.csv")
zcontrol = pd.read_csv("zcontrol.csv")

#mearging the data and a bit cleaning.
data = pd.concat([amb_temp, comp_temp, mcu_temp ,current ,door,energy ,power ,power_factor ,voltage ,zcontrol], axis = 1)
data.columns = ["date",'amb_temp', "date",'comp_temp', "date",'mcu_temp' ,"date",'current' ,"date",'door',"date",'energy' ,"date",'power' ,"date",'power_factor' ,"date",'voltage' ,"date",'zcontrol']
data = data[['amb_temp','comp_temp','mcu_temp' ,'current' ,'door','energy' ,'power' ,'power_factor' ,'voltage' ,'zcontrol']]
#data.fillna(data.mean, inplace=True)
doorarray = np.array(data["door"].values)
currentarray = np.array(data["current"].values)
amb_temparray = np.array(data["amb_temp"].values)
comp_temparray = np.array(data["comp_temp"].values)
mcu_temparray = np.array(data["mcu_temp"].values)
energyarray = np.array(data["energy"].values)
powerarray = np.array(data["power"].values)
power_factorarray = np.array(data["power_factor"].values)
voltagearray = np.array(data["voltage"].values)
zcontrolarray = np.array(data["zcontrol"].values)

In [3]:
xcurrent = tf.placeholder(shape=[], dtype=tf.float32, name = "xcurrent")
xprev = tf.placeholder(shape=[], dtype=tf.float32, name = "xprev")

# variable for Granger-Lasso
A = tf.Variable(1,dtype=tf.float32)

#Output is xcurrent = A*xprev
model_output = tf.multiply(xprev, A)

In [4]:
loss = tf.square(tf.abs(tf.subtract(xcurrent,tf.multiply(xprev,model_output))))

In [18]:
alpha = 0.0001
my_opt = tf.train.GradientDescentOptimizer(alpha)
train_step = my_opt.minimize(loss)

In [19]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [20]:
refdoor = []
for i in range(100):
    sess.run(train_step, feed_dict={xcurrent: doorarray[i+1], xprev:doorarray[i]}) 
    Adoor = sess.run(A)
    refdoor.append(Adoor)
    assign_op = A.assign(1)
    sess.run(assign_op)
    if i%20 == 0:
        print("xcurrent is ", doorarray[i+1]," xprev is ",doorarray[i]," A is ",Adoor)

xcurrent is  1  xprev is  1  A is  1.0
xcurrent is  1  xprev is  1  A is  1.0
xcurrent is  1  xprev is  1  A is  1.0
xcurrent is  1  xprev is  1  A is  1.0
xcurrent is  1  xprev is  1  A is  1.0


In [21]:
refcurrent = []
for i in range(100):
    sess.run(train_step, feed_dict={xcurrent: currentarray[i+1], xprev: currentarray[i]}) 
    Acurrent = sess.run(A)
    refcurrent.append(Acurrent)
    assign_op = A.assign(1)
    sess.run(assign_op)
    if i%20 == 0:
        print("xcurrent is ", currentarray[i+1]," xprev is ",currentarray[i]," A is ",Acurrent)

xcurrent is  10.20899963  xprev is  0.874000013  A is  1.00144
xcurrent is  0.870000005  xprev is  0.866999984  A is  1.00002
xcurrent is  0.875  xprev is  0.874000013  A is  1.00002
xcurrent is  0.875  xprev is  0.874000013  A is  1.00002
xcurrent is  0.870000005  xprev is  0.876999974  A is  1.00002


In [22]:
refpower = []
for i in range(100):
    sess.run(train_step, feed_dict={xcurrent: powerarray[i+1], xprev: powerarray[i]}) 
    Apower = sess.run(A)
    refpower.append(Apower)
    assign_op = A.assign(1)
    sess.run(assign_op)
    if i%20 == 0:
        print("xcurrent is ", powerarray[i+1]," xprev is ",powerarray[i]," A is ",Apower)

xcurrent is  2235.0  xprev is  209.0  A is  -362080.0
xcurrent is  207.0  xprev is  207.0  A is  -365432.0
xcurrent is  209.0  xprev is  209.0  A is  -379779.0
xcurrent is  210.0  xprev is  209.0  A is  -379770.0
xcurrent is  208.0  xprev is  210.0  A is  -387126.0


In [23]:
refpower_factor = []
for i in range(100):
    sess.run(train_step, feed_dict={xcurrent: power_factorarray[i+1], xprev: power_factorarray[i]}) 
    Apower_factor = sess.run(A)
    refpower_factor.append(Apower_factor)
    assign_op = A.assign(1)
    sess.run(assign_op)
    if i%20 == 0:
        print("xcurrent is ", power_factorarray[i+1]," xprev is ",power_factorarray[i]," A is ",Apower_factor)

xcurrent is  0.889999986  xprev is  0.970000029  A is  0.99999
xcurrent is  0.970000029  xprev is  0.970000029  A is  1.00001
xcurrent is  0.970000029  xprev is  0.970000029  A is  1.00001
xcurrent is  0.970000029  xprev is  0.970000029  A is  1.00001
xcurrent is  0.970000029  xprev is  0.970000029  A is  1.00001


In [24]:
#sess = tf.Session()
#init = tf.global_variables_initializer()
#sess.run(init)

In [25]:
refenergy = []
for i in range(100):
    sess.run(train_step, feed_dict={xcurrent: energyarray[i+1], xprev: energyarray[i]}) 
    Aenergy = sess.run(A)
    refenergy.append(Aenergy)
    assign_op = A.assign(1)
    sess.run(assign_op)
    if i%20 == 0:
        print("xcurrent is ", energyarray[i+1]," xprev is ", energyarray[i]," A is ", Aenergy)  

xcurrent is  132556.0  xprev is  132556.0  A is  -6.17482e+16
xcurrent is  132748.0  xprev is  132724.0  A is  -6.20618e+16
xcurrent is  132940.0  xprev is  132916.0  A is  -6.24217e+16
xcurrent is  133132.0  xprev is  133108.0  A is  -6.27832e+16
xcurrent is  133300.0  xprev is  133300.0  A is  -6.31462e+16


In [26]:
refvoltage = []
for i in range(100):
    sess.run(train_step, feed_dict={xcurrent: voltagearray[i+1], xprev: voltagearray[i]}) 
    Avoltage = sess.run(A)
    refvoltage.append(Avoltage)
    assign_op = A.assign(1)
    sess.run(assign_op)
    if i%20 == 0:
        print("xcurrent is ", voltagearray[i+1]," xprev is ", voltagearray[i]," A is ", Avoltage)

xcurrent is  244.5  xprev is  248.3999939  A is  -758423.0
xcurrent is  247.8000031  xprev is  248.0  A is  -753499.0
xcurrent is  248.1999969  xprev is  248.3000031  A is  -757154.0
xcurrent is  249.0  xprev is  248.6000061  A is  -760818.0
xcurrent is  249.0  xprev is  248.8999939  A is  -764504.0


In [27]:
refamb_temp = []
for i in range(100):
    sess.run(train_step, feed_dict={xcurrent: amb_temparray[i+1], xprev: amb_temparray[i]}) 
    Aamb_temp = sess.run(A)
    refamb_temp.append(Aamb_temp)
    assign_op = A.assign(1)
    sess.run(assign_op)
    if i%20 == 0:
        print("xcurrent is ", amb_temparray[i+1]," xprev is ", amb_temparray[i]," A is ", Aamb_temp) 

xcurrent is  27.937  xprev is  27.937  A is  -116.468
xcurrent is  28.0  xprev is  28.0  A is  -117.541
xcurrent is  28.187  xprev is  28.187  A is  -120.769
xcurrent is  28.25  xprev is  28.25  A is  -121.872
xcurrent is  28.5  xprev is  28.5  A is  -126.32


In [28]:
refcomp_temp = []
for i in range(100):
    sess.run(train_step, feed_dict={xcurrent: comp_temparray[i+1], xprev: comp_temparray[i]}) 
    Acomp_temp = sess.run(A)
    refcomp_temp.append(Acomp_temp)
    assign_op = A.assign(1)
    sess.run(assign_op)
    if i%20 == 0:
        print("xcurrent is ", comp_temparray[i+1]," xprev is ", comp_temparray[i]," A is ", Acomp_temp)   

xcurrent is  24.312  xprev is  24.562  A is  -68.8588
xcurrent is  24.687  xprev is  25.062  A is  -74.8017
xcurrent is  24.687  xprev is  25.875  A is  -85.3446
xcurrent is  25.437  xprev is  27.062  A is  -102.542
xcurrent is  26.625  xprev is  29.25  A is  -140.842


In [29]:
refmcu_temp = []
for i in range(100):
    sess.run(train_step, feed_dict={xcurrent: mcu_temparray[i+1], xprev: mcu_temparray[i]}) 
    Amcu_temp = sess.run(A)
    refmcu_temp.append(Amcu_temp)
    assign_op = A.assign(1)
    sess.run(assign_op)
    if i%20 == 0:
        print("xcurrent is ", mcu_temparray[i+1]," xprev is ", mcu_temparray[i]," A is ", Amcu_temp)

xcurrent is  9.312  xprev is  9.187  A is  -0.267516
xcurrent is  9.25  xprev is  9.062  A is  -0.196812
xcurrent is  9.125  xprev is  8.937  A is  -0.13008
xcurrent is  9.062  xprev is  8.812  A is  -0.0652112
xcurrent is  8.875  xprev is  8.625  A is  0.0252504


In [22]:
data.describe()

C:\Users\sharan\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,amb_temp,comp_temp,mcu_temp,current,door,energy,power,power_factor,voltage,zcontrol
count,23020.000000,23020.000000,23020.000000,23017.000000,23020.000000,22694.000000,23017.000000,23017.000000,23017.000000,23020.0
mean,29.888196,31.885758,9.059751,6.261266,0.909861,262060.419847,1389.522483,0.933945,246.338398,1.0
std,2.930325,4.706552,0.740041,4.976801,0.286387,77652.080031,1090.411178,0.034697,5.024132,0.0
min,23.812000,19.687000,7.625000,0.819000,0.000000,132556.000000,187.000000,0.840000,232.699997,1.0
25%,27.750000,28.062000,8.562000,NaN,1.000000,NaN,NaN,NaN,NaN,1.0
50%,29.437000,32.000000,8.937000,NaN,1.000000,NaN,NaN,NaN,NaN,1.0
75%,31.687000,35.375000,9.312000,NaN,1.000000,NaN,NaN,NaN,NaN,1.0
max,37.750000,44.062000,14.375000,19.261000,1.000000,399124.000000,2792.000000,0.980000,260.299988,1.0


In [23]:
# I have not used zcontrol because there is only one value in it, hence making it pointless.
data.zcontrol.unique()

array([1], dtype=int64)

In [32]:
matrix = np.matrix([refdoor, refcurrent, refpower, refpower_factor, refamb_temp, refcomp_temp, refmcu_temp, refenergy, refvoltage])

In [36]:
referancedata = pd.DataFrame(data = matrix.T)

In [37]:
referancedata.shape

(100, 9)

In [38]:
referancedata.columns = ["refdoor", "refcurrent", "refpower", "refpower_factor", "refamb_temp", "refcomp_temp", "refmcu_temp", "refenergy", "refvoltage"]

In [39]:
referancedata.head(20)

,refdoor,refcurrent,refpower,refpower_factor,refamb_temp,refcomp_temp,refmcu_temp,refenergy,refvoltage
0,1.0,1.001443,-3.620795e+05,0.999990,-116.467720,-68.858765,-0.267516,-6.174819e+16,-758422.8125
1,1.0,-0.950477,-4.988101e+09,1.000017,-116.467720,-65.644859,-0.341254,-6.174819e+16,-711813.8125
2,1.0,-1.335047,-6.159542e+09,1.000015,-116.467720,-105.783325,-0.382355,-6.179292e+16,-709482.8750
3,1.0,-1.260180,-5.942823e+09,1.000015,-116.477394,-159.313431,-0.310176,-6.183768e+16,-709480.5000
4,1.0,-1.195565,-5.781575e+09,1.000015,-115.418800,-192.605347,-0.171471,-6.183768e+16,-711812.6250
5,1.0,-1.391620,-5.724212e+09,1.000026,-115.418800,-208.624695,-0.010674,-6.188246e+16,-710610.5000
6,1.0,1.000017,-3.654324e+05,1.000005,-115.409157,-210.567368,0.079708,-6.188246e+16,-746222.6875
7,1.0,1.000017,-3.654324e+05,1.000005,-116.467720,-175.404785,0.053481,-6.188246e+16,-747434.8750
8,1.0,1.000018,-3.654238e+05,1.000005,-116.467720,-117.893600,-0.035516,-6.188246e+16,-745014.5625
9,1.0,1.000017,-3.725626e+05,1.000005,-116.467720,-83.656563,-0.130080,-6.192727e+16,-747436.0625
